# Preprocessing

### Imports

In [7]:
import sys
import os
import mne

# Ensure project root is in sys.path
sys.path.append(os.path.abspath(".."))

from src.patient import Patient, patient_from_filepath
from src.recording import Recording
from src.preprocessing import bandpass_filter, notch_filter
from src.utils.plotting import plot_signals_mne

## Patient
We can obtain patient information via a `file_path` to a recording

In [ ]:
file_path = "../data/sleep-telemetry/ST7101J0-PSG.edf"
patient: Patient = patient_from_filepath(file_path)
print(patient)

Extracting EDF parameters from /home/franko/Applied-Machine-Learning/Project/data/sleep-cassette/SC4102E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8573999  =      0.000 ... 85739.990 secs...


/home/franko/Applied-Machine-Learning/Project/src/patient.py:72: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(file_path, preload=True)
/home/franko/Applied-Machine-Learning/Project/src/patient.py:72: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(file_path, preload=True)
/home/franko/Applied-Machine-Learning/Project/src/patient.py:72: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(file_path, preload=True)


Patient #10: Age-26, Sex-Male


## Recording

From the filename we can obtain the patient number, type of study (cassette or telemetry) and the night (1 or 2)

In [9]:
recording: Recording = Recording(file_path)
print(recording)

Recording: Cassette, Patient-10, Night-2


## Plotting

In [10]:
# Load raw PSG data
raw = mne.io.read_raw_edf(recording.file_path, preload=True)

plot_signals_mne(recording=recording, raw=raw, annotations=True)

Extracting EDF parameters from /home/franko/Applied-Machine-Learning/Project/data/sleep-cassette/SC4102E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8573999  =      0.000 ... 85739.990 secs...


/tmp/ipykernel_1315548/2827028566.py:2: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(recording.file_path, preload=True)
/tmp/ipykernel_1315548/2827028566.py:2: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(recording.file_path, preload=True)
/tmp/ipykernel_1315548/2827028566.py:2: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(recording.file_path, preload=True)
/home/franko/Applied-Machine-Learning/Project/src/utils/plotting.py:33: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(anno)


Channels marked as bad:
none


In [11]:
# Only give specific channels to plot_signals_mne
channels = raw.info.ch_names[:2]
print ("Channels: ", channels)

plot_signals_mne(recording=recording, raw=raw, annotations=True, channels=channels)


Channels:  ['EEG Fpz-Cz', 'EEG Pz-Oz']


/home/franko/Applied-Machine-Learning/Project/src/utils/plotting.py:33: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(anno)


Channels marked as bad:
none


## Filtering
We can apply the bandpass and notch filters

In [14]:
# resample to 200Hz
raw = raw.resample(sfreq=200, npad="auto")

raw = bandpass_filter(raw, 0.5, 40, "EEG")
raw = bandpass_filter(raw, 0.5, 40, "EOG")
raw = bandpass_filter(raw, 10, 90, "EMG")

raw = notch_filter(raw, 50, "EEG")
raw = notch_filter(raw, 50, "EOG")
raw = notch_filter(raw, 50, "EMG")

plot_signals_mne(recording=recording, raw=raw, annotations=True)

Sampling frequency of the instance is already 200.0, returning unmodified.


2025-05-12 13:11:21,519 - INFO - preprocessing - Bandpass filter applied to EEG channels: 0.5-40 Hz
2025-05-12 13:11:28,372 - INFO - preprocessing - Bandpass filter applied to EOG channels: 0.5-40 Hz
2025-05-12 13:11:29,491 - INFO - preprocessing - Bandpass filter applied to EMG channels: 10-90 Hz
2025-05-12 13:11:30,769 - INFO - preprocessing - Notch filter applied to EEG channels at: 50 Hz
2025-05-12 13:11:36,914 - INFO - preprocessing - Notch filter applied to EOG channels at: 50 Hz
2025-05-12 13:11:38,022 - INFO - preprocessing - Notch filter applied to EMG channels at: 50 Hz
/home/franko/Applied-Machine-Learning/Project/src/utils/plotting.py:33: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(anno)


Channels marked as bad:
none
